<a href="https://colab.research.google.com/github/Fuenfgeld/TeamDataScDatenmanagementUndArchivierung/blob/main/AbelHodelinHernandezWoche2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [27]:
import pandas as pd
from functools import reduce
import sqlite3
from sqlite3 import Error
import csv

from google.colab import drive
drive.mount("/content/drive", force_remount=True)


Mounted at /content/drive


In [28]:
# list elements in directory
!ls /content/drive/MyDrive/csv_files/allergy/
material_path = '/content/drive/MyDrive/csv_files/allergy/'
patient_type = "allergy"

'careplans (1).gsheet'	 imaging_studies.csv	    payers.csv
 careplans.csv		 imaging_studies.gsheet     payer_transitions.csv
 careplans.gsheet	 immunizations.csv	    procedures.csv
 conditions.csv		 medications.csv	    procedures.gsheet
 devices.csv		'observations (1).gsheet'   providers.csv
 disease.csv		 observations.csv	    source_allergy_test.db
 disease.gsheet		 observations.gsheet	    supplies.csv
 encounters.csv		 organizations.csv
 encounters.gsheet	 patients.csv


In [29]:
# source database path
DB_FILE_PATH = f"{material_path}{patient_type}_test.db"
# folder contains csv files
csv_path = f"{material_path}"
# print(csv_path)

# DB_FILE_PATH, csv_path

In [30]:
# dictionary key: name of table, values: sql query to create table
sql_table = {}
sql_index = {}

In [31]:
sql_table["disease"] = """CREATE TABLE IF NOT EXISTS disease (
                           START DATE,
                           STOP DATE,
                           PATIENT STRING,
                           ENCOUNTER STRING,
                           CODE STRING,
                           DESCRIPTION STRING,
                           FOREIGN KEY (PATIENT)
                              REFERENCES patients (Id) 
                           FOREIGN KEY (Encounter)
                              REFERENCES encounters (Id)
                           
                       );"""
# kein Index für id, da die db erzeugt ein autematisch für die Primary Keys
# Die andere Indizes sind je nach praktischer Nutzung/Interese/Projekt
sql_index["disease_patient"] = """CREATE INDEX ix_disease_patient on disease(PATIENTS);"""
sql_index["disease_encounter"] = """CREATE INDEX ix_disease_encounter on disease(ENCOUNTER);"""
sql_index["disease_code"] = """CREATE INDEX ix_disease_code on disease(CODE);"""
sql_index["disease_description"] = """CREATE INDEX ix_disease_description on disease(DESCRIPTION);"""

In [32]:
sql_table["patients"] = """CREATE TABLE IF NOT EXISTS patients (
                           Id STRING PRIMARY KEY,
                           BIRTHDATE DATE,
                           DEATHDATE DATE,
                           SSN STRING,
                           DRIVERS STRING,
                           PASSPORT STRING,
                           PREFIX STRING,
                           FIRST STRING,
                           LAST STRING,
                           SUFFIX STRING,
                           MAIDEN STRING,
                           MARITAL STRING,
                           RACE STRING,
                           ETHNICITY STRING,
                           GENDER STRING,
                           BIRTHPLACE STRING,
                           ADDRESS STRING,
                           CITY STRING,
                           STATE STRING,
                           COUNTRY STRING,
                           ZIP STRING,
                           LAT INTEGER,
                           LON INTEGER,
                           HEALTHCARE_EXPENSES INTEGER,
                           HEALTHCARE_COVERAGE INTEGER
                       );"""
# kein Index für id, da die db erzeugt ein autematisch für die Primary Keys
# Die andere Indizes sind je nach praktischer Nutzung/Interese/Projekt
sql_index["patients_race"] = """CREATE INDEX ix_patients_race on patients(RACE);""" 
sql_index["patients_ethnicity"] = """CREATE INDEX ix_patients_ethnicity on patients(ETHNICITY);"""
sql_index["patients_city"] = """CREATE INDEX ixpatients_city on patients(CITY);"""
sql_index["patients_gender"] = """CREATE INDEX ix_patients_gender on patients(GENDER);"""
sql_index["patients_country"] = """CREATE INDEX ix_patients_gender on patients(COUNTRY);"""
sql_index["patients_zip"] = """CREATE INDEX ix_patients_zip on patients(ZIP);"""

In [33]:
sql_table["encounters"] = """CREATE TABLE IF NOT EXISTS encounters (
                           Id STRING PRIMARY KEY,
                           START DATE,
                           STOP DATE,
                           PATIENT STRING,
                           ORGANIZATIONS STRING,
                           PROVIDER STRING,
                           PAYER STRING,
                           ENCOUNTERCLASS STRING,
                           CODE STRING,
                           DESCRIPTION STRING,
                           BASE_ENCOUNTER_COST INTEGER,
                           TOTAL_CLAIM_COST INTEGER,
                           PAYER_COVERAGE INTEGER,
                           REASONCODE STRING,
                           REASONDESCRIPTION STRING,
                           FOREIGN KEY (PATIENT)
                              REFERENCES patients (Id) 
                       );"""
sql_index["encounters_patient"] = """CREATE INDEX ix_encounters_patients on encounters(PATIENT);""" 
sql_index["encounters_description"] = """CREATE INDEX ix_encounters_description on encounters(DESCRIPTION);"""
sql_index["encounters_code"] = """CREATE INDEX ix_encounters_code on encounters(CODE);"""

In [34]:
sql_table["careplans"] = """CREATE TABLE IF NOT EXISTS careplans (
                           Id STRING PRIMARY KEY,
                           START DATE,
                           STOP DATE,
                           PATIENT STRING,
                           ENCOUNTER STRING,
                           CODE STRING,
                           DESCRIPTION STRING,
                           REASONCODE STRING,
                           REASONDESCRIPTION STRING,
                           FOREIGN KEY (PATIENT)
                              REFERENCES patients (Id) 
                           FOREIGN KEY (Encounter)
                              REFERENCES encounters (Id) 

                       );"""
sql_index["careplans_patient"] = """CREATE INDEX ix_careplans_patients on careplans(PATIENT);"""
sql_index["careplans_encounters"] = """CREATE INDEX ix_careplans_encounters on careplans(ENCOUNTER);"""
sql_index["careplans_description"] = """CREATE INDEX ix_careplans_description on careplans(DESCRIPTION);"""
sql_index["careplans_code"] = """CREATE INDEX ix_careplans_code on careplans(CODE);"""
sql_index["careplans_reasondescription"] = """CREATE INDEX ix_careplans_reasondescription on careplans(REASONDESCRIPTION);"""
sql_index["careplans_reasoncode"] = """CREATE INDEX ix_careplans_reasoncode on careplans(REASONCODE);"""

In [35]:
sql_table["conditions"] = """CREATE TABLE IF NOT EXISTS conditions (
                           START DATE,
                           STOP DATE,
                           PATIENT STRING,
                           ENCOUNTER STRING,
                           CODE STRING,
                           DESCRIPTION STRING,
                           FOREIGN KEY (PATIENT)
                              REFERENCES patients (Id) 
                           FOREIGN KEY (Encounter)
                              REFERENCES encounters (Id) 

                       );"""
sql_index["conditions_patient"] = """CREATE INDEX ix_conditions_patient on conditions(PATIENT);"""
sql_index["conditions_encounter"] = """CREATE INDEX ix_conditions_encounter on conditions(ENCOUNTER);"""
sql_index["conditions_code"] = """CREATE INDEX ix_conditions_code on conditions(CODE);"""
sql_index["conditions_description"] = """CREATE INDEX ix_conditions_description on conditions(DESCRIPTION);"""

In [36]:
sql_table["procedures"]= """CREATE TABLE IF NOT EXISTS procedures (
                           DATE DATE,
                           PATIENT STRING,
                           ENCOUNTER STRING,
                           CODE STRING,
                           DESCRIPTION STRING,
                           BASE_COST INTEGER,
                           REASONCODE STRING,
                           REASONDESCRIPTION STRING,
                           FOREIGN KEY (PATIENT)
                              REFERENCES patients (Id) 
                           FOREIGN KEY (Encounter)
                              REFERENCES encounters (Id) 

                       );"""
sql_index["procedures_patient"] = """CREATE INDEX ix_procedures_patient on procedures(PATIENT);"""
sql_index["procedures_encounter"] = """CREATE INDEX ix_procedures_encounter on procedures(ENCOUNTER);"""
sql_index["procedures_code"] = """CREATE INDEX ix_procedures_code on procedures(CODE);"""
sql_index["procedures_description"] = """CREATE INDEX ix_procedures_description on procedures(DESCRIPTION);"""

In [37]:
sql_table["observations"]= """CREATE TABLE IF NOT EXISTS observations (
                           DATE DATE,
                           PATIENT STRING,
                           ENCOUNTER STRING,
                           CODE STRING,
                           DESCRIPTION STRING,
                           VALUE STRING,
                           UNITS STRING,
                           TYPE STRING,
                           FOREIGN KEY (PATIENT)
                              REFERENCES patients (Id) 
                           FOREIGN KEY (Encounter)
                              REFERENCES encounters (Id) 

                       );"""
sql_index["observations_patient"] = """CREATE INDEX ix_observations_patient on observations(PATIENT);"""
sql_index["observations_encounter"] = """CREATE INDEX ix_observations_encounter on observations(ENCOUNTER);"""
sql_index["observations_code"] = """CREATE INDEX ix_observations_code on observations(CODE);"""
sql_index["observations_description"] = """CREATE INDEX ix_observations_description on observations(DESCRIPTION);"""

In [38]:
sql_table["devices"] = """CREATE TABLE IF NOT EXISTS devices (
                           START DATE,
                           STOP DATE,
                           PATIENT STRING,
                           ENCOUNTER STRING,
                           CODE STRING,
                           DESCRIPTION STRING,
                           UDI STRING,
                           FOREIGN KEY (PATIENT)
                              REFERENCES patients (Id) 
                           FOREIGN KEY (Encounter)
                              REFERENCES encounters (Id)
                           
                       );"""
sql_index["devices_patient"] = """CREATE INDEX ix_devices_patient on devices(PATIENT);"""
sql_index["devices_encounter"] = """CREATE INDEX ix_devices_encounter on devices(ENCOUNTER);"""
sql_index["devices_code"] = """CREATE INDEX ix_devices_code on devices(CODE);"""
sql_index["devices_description"] = """CREATE INDEX ix_devices_description on devices(DESCRIPTION);"""

In [39]:
sql_table["imaging_studies"] = """CREATE TABLE IF NOT EXISTS imaging_studies (
                           Id STRING PRIMARY KEY,
                           DATE DATE,
                           PATIENT STRING,
                           ENCOUNTER STRING,
                           BODYSITE_CODE STRING,
                           BODYSITE_DESCRIPTION STRING,
                           MODALITY_CODE STRING,
                           MODALITY_DESCRIPTION STRING,
                           SOP_CODE STRING,
                           SOP_DESCRIPTION STRING,
                           FOREIGN KEY (PATIENT)
                              REFERENCES patients (Id) 
                           FOREIGN KEY (Encounter)
                              REFERENCES encounters (Id) 

                       );"""
sql_index["imaging_studies_patient"] = """CREATE INDEX ix_imaging_studies_patient on imaging_studies(PATIENT);"""
sql_index["imaging_studies_encounter"] = """CREATE INDEX ix_imaging_studies_encounter on imaging_studies(ENCOUNTER);"""
sql_index["imaging_studies_code"] = """CREATE INDEX ix_imaging_studies_code on imaging_studies(CODE);"""
sql_index["imaging_studies_description"] = """CREATE INDEX ix_imaging_studies_description on imaging_studies(DESCRIPTION);"""

In [40]:
sql_table["immunizations"]= """CREATE TABLE IF NOT EXISTS immunizations(
                           DATE DATE,
                           PATIENT STRING,
                           ENCOUNTER STRING,
                           CODE STRING,
                           DESCRIPTION STRING,
                           BASE_COST INTEGER,
                           FOREIGN KEY (PATIENT)
                              REFERENCES patients (Id) 
                           FOREIGN KEY (Encounter)
                              REFERENCES encounters (Id) 

                       );"""
sql_index["immunizations_patient"] = """CREATE INDEX ix_immunizations_patient on immunizations(PATIENT);"""
sql_index["immunizations_encounter"] = """CREATE INDEX ix_immunizations_encounter on immunizations(ENCOUNTER);"""
sql_index["immunizations_code"] = """CREATE INDEX ix_immunizations_code on immunizations(CODE);"""
sql_index["immunizations_description"] = """CREATE INDEX ix_immunizations_description on immunizations(DESCRIPTION);"""